In [ ]:
import cohere, os
co = cohere.Client(os.getenv("COHERE_API_KEY"))

def generate_query_from_promt(prompt):
    response = co.chat(
        message=prompt,
        search_queries_only=True
    )
    return response.search_queries[0]["text"]

In [ ]:
# Example of how to generate a query from a prompt
generate_query_from_promt("Where do the tallest penguins live?")

In [ ]:
import weaviate, os

client = weaviate.connect_to_wcs(
    cluster_url=os.getenv("WORKSHOP_DEMO_URL"),
    auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WORKSHOP_DEMO_KEY_READ")),

    headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_API_KEY"),
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY"),

        "X-AWS-Access-Key": os.getenv("AWS_ACCESS_KEY"),
        "X-AWS-Secret-Key": os.getenv("AWS_SECRET_KEY"),
    },

    additional_config=weaviate.config.AdditionalConfig(timeout=(500, 1500))
)

client.is_ready()

In [ ]:
from weaviate.classes.query import Filter

def two_step_rag(user_prompt):
    # Step 1
    prompt = user_prompt + " Please only use the provided content with this prompt. Don't make things up."
    
    generated_query = generate_query_from_promt(prompt)
    print("=== Generated Query ===")
    print(f"Generated query: {generated_query}")

    # Step 2
    wiki = client.collections.get("Wikipedia")

    response = wiki.generate.near_text(
        query=generated_query,
        auto_limit=1,
        grouped_task=prompt,
        grouped_properties=["text", "title"]
    )

    # Print results
    print("=== Generated Response ===")
    print(response.generated)

    print("=== Source ===")
    for item in response.objects:
        print(item.properties)

In [ ]:
# two_step_rag("What wild animals do we know about?")
two_step_rag("How do airplanes fly?")

In [ ]:
two_step_rag("What are the pros and cons of automation using computer?")

In [ ]:
two_step_rag("How do CPUs work?")